## 1. Importación de librerías

In [ ]:
import pandas as pd
import re

## 2. Cargar archivos

In [ ]:
df = pd.read_csv("../output/dataset_farmacia_unificado.csv")
print(f"Dataset original: {df.shape[0]} filas x {df.shape[1]} columnas")
df.head(5)

## 3. Normalización de nombres de medicamento

In [ ]:
def normalizar_nombres(nombre):
    if pd.isna(nombre):
        return nombre

    nombre = nombre.lower().strip()

    nombre = re.sub(r'(\d+)\s*(mg|ml|gr|g|%)', r'\1 \2', nombre)
    nombre = re.sub(r'x\s*(\d+)', r'\1 unidades ', nombre)

    replacements = {
        "comp": "comprimidos",
        "comp.": "comprimidos",
        "caps": "capsulas",
        "cap": "capsulas",
        "tab": "tabletas",
        "tabl": "tabletas",
        "amp": "ampollas",
        "inj": "inyectable",
        "eferv": "efervescente",
        "sr": "liberacion sostenida",
        "sl": "sublingual",
        "nf": "nueva formula",
        "duo": "dual"
    }

    for k, v in replacements.items():
        nombre = re.sub(rf'\b{k}\b', v, nombre)

    nombre = re.sub(r'[^a-z0-9\s%]', ' ', nombre)

    nombre = re.sub(r'\s+', ' ', nombre).strip()

    return nombre

df["nombre_normalizado"] = df["nombre"].apply(normalizar_nombres)

df[["nombre", "nombre_normalizado"]].head(10)

## 4. Crear columna de cantidad total

In [ ]:
if 'fraccion' in df.columns:
    df['cantidad_total'] = df['cantidad'] + df['fraccion'].fillna(0)
    df = df.drop(columns=['cantidad', 'fraccion'])
else:
    df['cantidad_total'] = df['cantidad']
    
df.head(5)



## 5. Eliminar columnas ID

In [ ]:
id_cols = [
    'auto_detalle', 'venta', 'compra2',
    'compra', 'medicamento', 'auto_med', 'auto'
]
df = df.drop(columns=id_cols, errors='ignore')

df.head(3)



## 6. Renombrar columnas

In [ ]:
df = df.rename(columns={
    'auto_venta': 'id_venta',
    'monto_recibido': 'total_venta',
    'total': 'total_unitario',
    'precio': 'precio_unitario'
})

df.head(3)

,id_venta,fecha,total_venta,precio_unitario,total_unitario,precio_compra,nombre,nombre_normalizado,cantidad_total
0,1,2019-01-07,0.0,8.0,16.0,4.75,URIDON F X 16 COMPRIMIDOS,uridon f 16 unidades comprimidos,2.0
1,17,2019-01-07,200.0,0.8,44.0,0.46,METFORMINA SAE 850mg x 100com,metformina sae 850 mg 100 unidades com,55.0
2,17,2019-01-07,200.0,6.0,6.0,3.80,ALCOHOL 120 ML,alcohol 120 ml,1.0


## 7. Exportar resultado

In [ ]:
print("\nTipos de datos finales:")
print(df.dtypes)

print("\nNulos por columna:")
print(df.isnull().sum())

print(f"\nDataset final: {df.shape[0]} filas x {df.shape[1]} columnas")

print("\nPrimeras filas:")
df.head(10)



Tipos de datos finales:
id_venta                int64
fecha                  object
total_venta           float64
precio_unitario       float64
total_unitario        float64
precio_compra         float64
nombre                 object
nombre_normalizado     object
cantidad_total        float64
dtype: object

Nulos por columna:
id_venta              0
fecha                 0
total_venta           0
precio_unitario       0
total_unitario        0
precio_compra         0
nombre                0
nombre_normalizado    0
cantidad_total        0
dtype: int64

Dataset final: 175352 filas x 9 columnas

Primeras filas:


,id_venta,fecha,total_venta,precio_unitario,total_unitario,precio_compra,nombre,nombre_normalizado,cantidad_total
0,1,2019-01-07,0.0,8.00,16.0,4.75,URIDON F X 16 COMPRIMIDOS,uridon f 16 unidades comprimidos,2.0
1,17,2019-01-07,200.0,0.80,44.0,0.46,METFORMINA SAE 850mg x 100com,metformina sae 850 mg 100 unidades com,55.0
2,17,2019-01-07,200.0,6.00,6.0,3.80,ALCOHOL 120 ML,alcohol 120 ml,1.0
3,17,2019-01-07,200.0,0.45,12.6,0.34,ASPIRINETA 100 (BAYER),aspirineta 100 bayer,28.0
4,17,2019-01-07,200.0,1.00,20.0,0.66,JERINGA 1 ML INSULINA NIPRO,jeringa 1 ml insulina nipro,20.0
5,17,2019-01-07,200.0,2.00,4.0,1.00,ESPIRONOLACTONA 100MG X 100COMP (DISMEDIN),espironolactona 100 mg 100 unidades comprimido...,2.0
6,17,2019-01-07,200.0,2.00,12.0,1.44,SUPOSITORIO LARGO DE GLICERINA x 50 supost,supositorio largo de glicerina 50 unidades supost,6.0
7,17,2019-01-07,200.0,0.50,5.0,0.30,REINA OBRERA MIEL sabores,reina obrera miel sabores,10.0
8,17,2019-01-07,200.0,23.00,23.0,18.07,TOLESTAN 85mg x 2 comp,tolestan 85 mg 2 unidades comprimidos,1.0
9,17,2019-01-07,200.0,25.00,25.0,12.50,OFTALMOL GENTA 10ML,oftalmol genta 10 ml,1.0


In [ ]:
output_path = "../output/dataset_farmacia_integrado.csv"
df.to_csv(output_path, index=False)
print(f"Dataset integrado guardado como: {output_path}")
print(f"Forma del dataset agrupado: {df.shape[0]} filas x {df.shape[1]} columnas")


Dataset integrado guardado como: ../output/dataset_farmacia_integrado.csv
Forma del dataset agrupado: 175352 filas x 9 columnas
